In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
import math

In [ ]:
iris = load_iris()
#iris.data -> features, iris.target -> labels
X = iris.data
y = iris.target

In [ ]:
# split into test and train dataset, and use random_state=48
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=48)

In [ ]:
from sklearn.preprocessing import StandardScaler
#ss = StandardScaler()
#X_train=ss.fit_transform(X_train)
#X_test=ss.transform(X_test)

In [ ]:
trainSet=np.column_stack((X_train, y_train))
testSet=np.column_stack((X_test, y_test))

In [ ]:
# p=1 : return Manhattan Distance
# p=2 : return Eucludean Distance
# w=1 : Normal
# w=2 : weighted
def KNN(p,trainSet, testSet, k, w):
    acc=0
    for i in range(len(testSet)):
        indexing=[]
        distance=[]
        
        if p ==1:
            for q in range(len(trainSet)):
                indexing.append(q)
                distance.append(abs(trainSet[q,:4]-testSet[i,:4]).sum())
        elif p ==2 :
            for q in range(len(trainSet)):
                indexing.append(q)
                distance.append(np.sqrt(((trainSet[q,:4]-testSet[i,:4])**2).sum()))
                
        neighbor=pd.Series(distance,indexing)
        neighbor.sort_values(inplace=True)
        neighbor=neighbor.iloc[:k]
        
        if w ==1 :    
            pred= pd.DataFrame(trainSet).iloc[list(neighbor.index.astype(int)),-1].mode()[0]
        elif w==2 :
            pred_df=pd.concat([np.reciprocal(neighbor)/(np.reciprocal(neighbor).sum()),pd.DataFrame(trainSet).iloc[list(neighbor.index.astype(int)),-1]],axis=1)
            pred_df.columns=['weight','category']
            pred= (pred_df.groupby('category').sum().reset_index().sort_values(by='weight',ascending=False)).iloc[0,0]
            
        
        print(str(i) + ' > predicted : ' + str(pred) + ', actual : ' + str(testSet[i][-1]))
        if pred == testSet[i][-1]:
            acc+=1
    print('Accuracy: ' + str(acc/len(testSet)) + '%')

In [ ]:
KNN(1,trainSet,testSet,3,1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

neigh = KNeighborsClassifier(n_neighbors=3,p=1)
neigh.fit(trainSet[:,:4],trainSet[:,-1])
pred=neigh.predict(testSet[:,:4])
display(accuracy_score(pred,testSet[:,4])) # 동일

In [ ]:
KNN(2,trainSet,testSet,3,1)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3,p=2)
neigh.fit(trainSet[:,:4],trainSet[:,-1])
pred=neigh.predict(testSet[:,:4])
display(accuracy_score(pred,testSet[:,4])) # 동일

In [ ]:
dis_list=[1,2]
num_k_list=[3,5,7,9]
weight_list=[1,2]

In [ ]:
from itertools import product

items = [dis_list,num_k_list,weight_list]
params=list(product(*items))

In [ ]:
scores=[]
dis=[]
num_k=[]
weight=[]

def tune_KNN(p,trainSet, testSet, k, w):
    acc=0
    for i in range(len(testSet)):
        indexing=[]
        distance=[]
        
        if p ==1:
            for q in range(len(trainSet)):
                indexing.append(q)
                distance.append(abs(trainSet[q,:4]-testSet[i,:4]).sum())
        elif p ==2 :
            for q in range(len(trainSet)):
                indexing.append(q)
                distance.append(np.sqrt(((trainSet[q,:4]-testSet[i,:4])**2).sum()))
                
        neighbor=pd.Series(distance,indexing)
        neighbor.sort_values(inplace=True)
        neighbor=neighbor.iloc[:k]
        
        if w ==1 :    
            pred= pd.DataFrame(trainSet).iloc[list(neighbor.index.astype(int)),-1].mode()[0]
        elif w==2 :
            pred_df=pd.concat([np.reciprocal(neighbor)/(np.reciprocal(neighbor).sum()),pd.DataFrame(trainSet).iloc[list(neighbor.index.astype(int)),-1]],axis=1)
            pred_df.columns=['weight','category']
            pred= (pred_df.groupby('category').sum().reset_index().sort_values(by='weight',ascending=False)).iloc[0,0]
            
        if pred == testSet[i][-1]:
            acc+=1
    dis.append(p)
    num_k.append(k)
    weight.append(w)
    scores.append(acc/len(testSet)) 
for i in params:
    p,k,w=i
    tune_KNN(p,trainSet,testSet,k,w)

In [ ]:
score=pd.DataFrame([dis,num_k,weight,scores]).T
score.columns=['d','k','w','score']

In [ ]:
score.sort_values(by='score',ascending=False)
# 각 경우의 수별 score